In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.utils import save_image
import os
import random
from PIL import Image
import numpy as np
from tqdm import tqdm
import json
from matplotlib import pyplot as plt
from torch.cuda.amp import autocast, GradScaler

In [2]:
class SketchDataset(Dataset):
    def __init__(self, bad_sketch_dir, good_sketch_dir):
        self.bad_sketch_paths = sorted(
            [os.path.join(bad_sketch_dir, x) for x in os.listdir(bad_sketch_dir)]
        )
        self.good_sketch_paths = sorted(
            [os.path.join(good_sketch_dir, x) for x in os.listdir(good_sketch_dir)]
        )
        self.transform = transforms.Compose(
            [
                transforms.Grayscale(),
                transforms.Resize((256, 256)),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )

        self.transform_augment = transforms.Compose(
            [
                transforms.Grayscale(num_output_channels=1),
                transforms.RandomHorizontalFlip(),
                transforms.RandomAffine(10, shear=10, scale=(0.8, 1.2), fill=255),
                transforms.ToTensor(),
                transforms.Normalize((0.5,), (0.5,)),
            ]
        )

    def __getitem__(self, index):
        bad_sketch = Image.open(
            self.bad_sketch_paths[index % len(self.bad_sketch_paths)]
        )

        good_sketch = Image.open(random.choice(self.good_sketch_paths))

        bad_sketch = self.transform(bad_sketch)
        good_sketch = self.transform(good_sketch)

        return {"A": bad_sketch, "B": good_sketch}

    def __len__(self):
        return len(self.bad_sketch_paths)

In [3]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.InstanceNorm2d(channels),
        )

    def forward(self, x):
        return x + self.block(x)


class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        # Initial convolution block
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=1, padding=3),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),
        )

        # Downsampling blocks
        self.down_blocks = nn.Sequential(
            self._make_layer(64, 128), self._make_layer(128, 256)
        )

        # Residual blocks
        self.res_blocks = nn.Sequential(*[ResidualBlock(256) for _ in range(9)])

        # Upsampling blocks
        self.up_blocks = nn.Sequential(
            self._make_layer(256, 128, upsample=True),
            self._make_layer(128, 64, upsample=True),
        )

        # Output convolution
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 1, kernel_size=7, stride=1, padding=3), nn.Tanh()
        )

    def _make_layer(self, in_channels, out_channels, upsample=False):
        if upsample:
            return nn.Sequential(
                nn.ConvTranspose2d(
                    in_channels, out_channels, 3, stride=2, padding=1, output_padding=1
                ),
                nn.InstanceNorm2d(out_channels),
                nn.ReLU(inplace=True),
            )
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, stride=2, padding=1),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.down_blocks(x)
        x = self.res_blocks(x)
        x = self.up_blocks(x)
        return self.conv2(x)


# Discriminator Network
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, padding=1),
            nn.InstanceNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, 4, padding=1),
        )

    def forward(self, x):
        return self.model(x)

In [4]:
# Training class for each sketch category
class SketchEnhancer:
    def __init__(
        self,
        class_name,
        train_set,
        val_set,
        test_set,
        batch_size=64,
        device="cuda",
        log_dir="logs",
    ):
        self.class_name = class_name
        self.device = device
        self.log_dir = os.path.join(log_dir, class_name)
        os.makedirs(self.log_dir, exist_ok=True)

        # Initialize networks
        self.G_AB = Generator().to(device)
        self.G_BA = Generator().to(device)
        self.D_A = Discriminator().to(device)
        self.D_B = Discriminator().to(device)

        # Initialize optimizers
        self.g_optimizer = optim.Adam(
            list(self.G_AB.parameters()) + list(self.G_BA.parameters()),
            lr=0.0002,
            betas=(0.5, 0.999),
        )
        self.d_optimizer = optim.Adam(
            list(self.D_A.parameters()) + list(self.D_B.parameters()),
            lr=0.0002,
            betas=(0.5, 0.999),
        )

        # Loss functions
        self.criterion_GAN = nn.MSELoss()
        self.criterion_cycle = nn.L1Loss()
        self.criterion_identity = nn.L1Loss()

        self.train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
        self.val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=True)
        self.test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

        self.g_scaler = GradScaler()
        self.d_scaler = GradScaler()

        # Initialize training history
        self.history = {
            "train_losses": [],
            "val_losses": [],
            "best_val_loss": float("inf"),
        }

    def compute_losses(self, batch, use_amp=False):
        real_A = batch["A"].to(self.device)
        real_B = batch["B"].to(self.device)

        with autocast(enabled=use_amp):
            # Generate fake samples
            fake_B = self.G_AB(real_A)
            fake_A = self.G_BA(real_B)

            # Reconstruct samples
            rec_A = self.G_BA(fake_B)
            rec_B = self.G_AB(fake_A)

            # Identity mapping
            id_A = self.G_BA(real_A)
            id_B = self.G_AB(real_B)

            # Calculate losses
            loss_id_A = self.criterion_identity(id_A, real_A) * 5.0
            loss_id_B = self.criterion_identity(id_B, real_B) * 5.0

            # GAN losses
            pred_fake_B = self.D_B(fake_B)
            loss_GAN_AB = self.criterion_GAN(pred_fake_B, torch.ones_like(pred_fake_B))
            pred_fake_A = self.D_A(fake_A)
            loss_GAN_BA = self.criterion_GAN(pred_fake_A, torch.ones_like(pred_fake_A))

            # Cycle losses
            loss_cycle_A = self.criterion_cycle(rec_A, real_A) * 10.0
            loss_cycle_B = self.criterion_cycle(rec_B, real_B) * 10.0

            # Total losses
            loss_G = (
                loss_GAN_AB
                + loss_GAN_BA
                + loss_cycle_A
                + loss_cycle_B
                + loss_id_A
                + loss_id_B
            )

            # Discriminator losses
            pred_real_A = self.D_A(real_A)
            loss_D_real_A = self.criterion_GAN(
                pred_real_A, torch.ones_like(pred_real_A)
            )
            pred_real_B = self.D_B(real_B)
            loss_D_real_B = self.criterion_GAN(
                pred_real_B, torch.ones_like(pred_real_B)
            )

            pred_fake_A = self.D_A(fake_A.detach())
            loss_D_fake_A = self.criterion_GAN(
                pred_fake_A, torch.zeros_like(pred_fake_A)
            )
            pred_fake_B = self.D_B(fake_B.detach())
            loss_D_fake_B = self.criterion_GAN(
                pred_fake_B, torch.zeros_like(pred_fake_B)
            )

            loss_D = (loss_D_real_A + loss_D_fake_A) * 0.5 + (
                loss_D_real_B + loss_D_fake_B
            ) * 0.5

        return {
            "loss_G": loss_G,
            "loss_D": loss_D,
            "loss_cycle": loss_cycle_A + loss_cycle_B,
            "loss_identity": loss_id_A + loss_id_B,
            "samples": (real_A, fake_B, rec_A),
        }

    def validate(self):
        self.G_AB.eval()
        self.G_BA.eval()
        self.D_A.eval()
        self.D_B.eval()

        val_losses = []
        with torch.no_grad(), autocast():
            for batch in self.val_loader:
                losses = self.compute_losses(batch, use_amp=True)
                val_losses.append(losses["loss_G"].item() + losses["loss_D"].item())

        return np.mean(val_losses)

    def train(self, num_epochs):
        # Create progress bars

        for epoch in range(num_epochs):
            # Training phase
            self.G_AB.train()
            self.G_BA.train()
            self.D_A.train()
            self.D_B.train()

            train_losses = []
            batch_bar = tqdm(self.train_loader, desc=f"Epoch {epoch+1}", leave=False)

            for batch in batch_bar:
                # Compute losses with AMP
                losses = self.compute_losses(batch, use_amp=True)

                # Update generators with AMP
                self.g_optimizer.zero_grad()
                self.g_scaler.scale(losses["loss_G"]).backward()
                self.g_scaler.step(self.g_optimizer)
                self.g_scaler.update()

                # Update discriminators with AMP
                self.d_optimizer.zero_grad()
                self.d_scaler.scale(losses["loss_D"]).backward()
                self.d_scaler.step(self.d_optimizer)
                self.d_scaler.update()

                # Update progress bar
                batch_bar.set_postfix(
                    {
                        "G_loss": f"{losses['loss_G'].item():.4f}",
                        "D_loss": f"{losses['loss_D'].item():.4f}",
                    }
                )

                train_losses.append(losses["loss_G"].item() + losses["loss_D"].item())

            # Validation phase
            val_loss = self.validate()

            # Update training history
            avg_train_loss = np.mean(train_losses)
            self.history["train_losses"].append(avg_train_loss)
            self.history["val_losses"].append(val_loss)

            # Update progress bar
            print(
                f"Epoch[{epoch+1}] Train Loss: {avg_train_loss:.4f}   Validation Loss: {val_loss:.4f}"
            )

            # Save best model
            if val_loss < self.history["best_val_loss"]:
                self.history["best_val_loss"] = val_loss
                self.save_models(best=True)

            # Save sample images
            if (epoch + 1) % 5 == 0:
                self.save_samples(*losses["samples"], epoch)

            # Save training history
            self.save_history()

    def save_samples(self, real_A, fake_B, rec_A, epoch):
        save_image(
            torch.cat([real_A, fake_B, rec_A], 0),
            f"samples_{self.class_name}_epoch_{epoch}.png",
            normalize=True,
        )

    def save_history(self):
        with open(os.path.join(self.log_dir, "training_history.json"), "w") as f:
            json.dump(self.history, f)

    def save_models(self, best=False):
        prefix = "best_" if best else ""
        torch.save(
            self.G_AB.state_dict(), os.path.join(self.log_dir, f"{prefix}G_AB.pth")
        )
        torch.save(
            self.G_BA.state_dict(), os.path.join(self.log_dir, f"{prefix}G_BA.pth")
        )
        torch.save(
            self.D_A.state_dict(), os.path.join(self.log_dir, f"{prefix}D_A.pth")
        )
        torch.save(
            self.D_B.state_dict(), os.path.join(self.log_dir, f"{prefix}D_B.pth")
        )

    def test(self):
        self.G_AB.eval()
        self.G_BA.eval()
        self.D_A.eval()
        self.D_B.eval()

        test_losses = []
        with torch.no_grad(), autocast():
            for batch in self.test_loader:
                losses = self.compute_losses(batch, use_amp=True)
                test_losses.append(losses["loss_G"].item() + losses["loss_D"].item())

        return np.mean(test_losses)

    def load_models(self):
        self.G_AB.load_state_dict(torch.load(f"G_AB_{self.class_name}.pth"))
        self.G_BA.load_state_dict(torch.load(f"G_BA_{self.class_name}.pth"))
        self.D_A.load_state_dict(torch.load(f"D_A_{self.class_name}.pth"))
        self.D_B.load_state_dict(torch.load(f"D_B_{self.class_name}.pth"))


# Main training loop for all classes
def train_all_classes(base_dir, classes, batch_size=64, num_epochs=100):
    for class_name in classes:
        print(f"\nTraining class: {class_name}")

        # Setup data paths
        bad_sketch_dir = os.path.join(base_dir, "bad_sketches", class_name)
        good_sketch_dir = os.path.join(base_dir, "good_sketches", class_name)

        # Create dataset and dataloader
        dataset = SketchDataset(bad_sketch_dir, good_sketch_dir)
        print(f"Number of samples: {len(dataset)}")
        test_split = 0.2
        val_split = 0.1

        train_set, test_set = random_split(
            dataset,
            [
                round(len(dataset) * (1 - test_split)),
                round(len(dataset) * test_split),
            ],
        )
        train_set, val_set = random_split(
            train_set,
            [
                round(len(train_set) * (1 - val_split)),
                round(len(train_set) * val_split),
            ],
        )
        print(f"Train set: {len(train_set)}")
        print(f"Validation set: {len(val_set)}")
        print(f"Test set: {len(test_set)}")

        # Initialize and train model
        enhancer = SketchEnhancer(class_name, train_set, val_set, test_set, batch_size)
        enhancer.train(num_epochs)
        enhancer.save_models()

        # Test model
        test_loss = enhancer.test()
        print(f"Test loss: {test_loss:.4f}")

In [ ]:
base_dir = "./Datasets/"
classes = os.listdir(os.path.join(base_dir, "bad_sketches"))

train_all_classes(base_dir, classes, batch_size=16, num_epochs=50)


Training class: airplane
Number of samples: 3001
Train set: 2161
Validation set: 240
Test set: 600


Epoch[1] Train Loss: 4.8358   Validation Loss: 1.5551


Epoch[2] Train Loss: 1.8084   Validation Loss: 1.8282


Epoch[3] Train Loss: 1.7786   Validation Loss: 2.6143


Epoch[4] Train Loss: 1.8100   Validation Loss: 1.7792


...

Epoch[28] Train Loss: 1.9870   Validation Loss: 1.7968


Epoch[29] Train Loss: 2.0538   Validation Loss: 1.5439


Epoch[30] Train Loss: 2.1141   Validation Loss: 2.3898


Epoch[31] Train Loss: 2.0941   Validation Loss: 2.3584


Epoch 32:  51%|█████▏    | 70/136 [01:55<01:48,  1.65s/it, G_loss=1.6980, D_loss=0.0906]